## MPO HGA Blockage

This is a Python Jupyter Notebook to illustrate how to visualize the blockage that suffers the High Gain Antenna due to the MPO structure. 

First thing we will do is indicate that we want the Python package matplotlib to be output in the notebook and to import the SpiceyPy package to use SPICE. We also ensure that project_path module is include, this will allow us get access to common packages and resources

In [ ]:
from project_path import data_path
import spiceypy as cspice
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import trange

We will setup the kernel set for the spice library, that will be used along the notebook. Note that if you are in **Datalabs** infrastructure and the SPICE volume is mounted, the latest kernel repositories are available. Additionally the *ess.datalabs* package contains many helpers and shortcuts.

Using the *data_path* shortcut we can get resources stored in the **ess-jupyternb/data** folder

In [ ]:
from ess.datalabs import get_local_metakernel, is_spice_volume_mounted

cspice.kclear()
if is_spice_volume_mounted():
    cspice.furnsh(get_local_metakernel('BEPICOLOMBO', 'bc_plan.tm'))
else:
   cspice.furnsh('/Users/randres/git/spice/bepicolombo/kernels/mk/bc_plan.tm')

cspice.furnsh(
    os.path.join(data_path, 'test', 'bc_mpo_hga_schulte_vector_test_v01.bc'))
cspice.furnsh(
    os.path.join(data_path, 'test', 'bc_mpo_sc_bus_v03.bds'))

The *bc_mpo_hga_schulte_vector_test_v01.bc* contains a full rotation of the schulte vector. It contains the attitude covering in 3600 seconds the full excursion along 360 degrees. The start point in 2021-01-01T00:00:00Z

In [ ]:
time_step_seconds = 10 # time step
et0 = cspice.utc2et('2021-01-01T00:00:00')
et = et0
rotation_vectors = []
for j in range(-180,181):
    M = cspice.pxform('MPO_HGA', 'MPO_SPACECRAFT', et)
    rotation_vectors.append(cspice.mxv(M, [0, 0, 1])) 
    et += time_step_seconds
rv = np.array(rotation_vectors)
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(rv[:,0],rv[:,1],rv[:,2]);
cspice.et2utc(et, 'ISOC', 0)


Now we calculate the blockage map with the help of this schulte vector. We simulate the exclusion for the dish of the antenna.

**Warning** It can take some minutes to get the map

In [ ]:


def discretized_dish(r, t):
    for i in range(len(r)):
       for j in range(t[i]):
        yield r[i], j*(2 * np.pi / t[i])


R = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
T = [1, 5, 10, 15, 20, 30, 40]

time_step_seconds = 10 # time step
angular_step_degrees = 12 
et0 = cspice.utc2et('2021-01-01T00:00:00')
el_angles = np.arange(-5.0, 145, 1)
az_angles = np.arange(-180, 181, 1)
coverage_map = np.zeros((el_angles.shape[0], az_angles.shape[0]))
et = et0
output = []
output_b = []
row_counter = el_angles.shape[0]
for i in trange(el_angles.shape[0]):
    for j in range(az_angles.shape[0]):
        el = el_angles[i]
        az = az_angles[j]
        rs = []
        ds = []
        for r, t in discretized_dish(R, T):
            r0 = np.array([r * np.cos(t), r * np.sin(t), 0]) / 1000
            r_opt = cspice.spkpos('MPO_HGA_OPT_EL', et, 'MPO_SPACECRAFT', 'NONE', 'MPO_SPACECRAFT')[0]
            M = cspice.pxform('MPO_HGA', 'MPO_SPACECRAFT', et)
            r = cspice.mxv(M, r0) + r_opt
            d = cspice.mxv(M, [0, 0, 1])
            rs.append(r)
            ds.append(d)
            
        xarray, flag_array = cspice.dskxv(False, 'MPO_SPACECRAFT', [-121000], et, 'MPO_SPACECRAFT',
                         rs, ds)
        
        coverage_map[i, j] += len(list(filter(lambda x: x, flag_array)))           
        et += time_step_seconds



The **plot_earth** function plots the projection in HGA frame of the earth path during the period passed as argument

In [ ]:
def plot_earth(utc0, utcf, color, label):
    et0 = cspice.utc2et(utc0)
    etf = cspice.utc2et(utcf)
    times = np.linspace(et0, etf, 1000)
    earth_radec = []
    for et in times:
        r_earth = cspice.spkpos('EARTH', et, 'MPO_HGA', 'NONE', 'MPO')[0]
        radec = np.rad2deg(cspice.recazl(r_earth, True, True))
        ra = (radec[1] - 360) if radec[1] > 180 else radec[1]
        dec = radec[2] + 90
        earth_radec.append([ra, dec])
    
    earth_radec = np.asarray(earth_radec)
    plt.plot(earth_radec[:, 0], earth_radec[:, 1], lw=1, color=color, label=label)
    return

We can plot also other pre-calculated masks:
* simplified: a rough estimation of the mask
* astrium: industry provide mask

In [ ]:
simplified = {
    'name': 'simplified',
    'az': [-180, -105,   0,  40,  60, 110, 135, 180],
    'el': [ 145,   90, 130, 130, 110, 110, 145, 145]
}

astrium = {
    'name': 'astrium',
    'az': [-180, -175, -170, -165, -160, -155, -150, -145, -140, -135, -130, 
           -125, -120, -115, -110, -105, -100, -95, -90, -85, -80, -75, -70, 
           -65, -60, -55, -50, -45, -40, -35, -30, -25, -20, -15, -10, -5, 0,
            5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85,
            90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155,
            160, 165, 170, 175, 180],
    'el': [144, 144, 144, 141.5, 135.5, 129.5, 126, 123, 119.5, 114.5, 110.5, 
           105, 99.5, 95.5, 93, 92, 91.5, 91.5, 91.5, 92.5, 94, 95.5, 100, 104, 
           110, 113, 114.5, 116.5, 120, 120.5, 121, 122.5, 126.5, 133, 134.5, 
           135, 135.5, 133, 130, 128.5, 129.5, 128.5, 128, 128.5, 130.5, 125.5, 
           120, 116, 113.5, 111.5, 110.5, 110, 110, 110, 109.5, 109, 109, 110, 
           113.5, 120, 127, 134, 140, 144, 144, 144, 144, 144, 144, 144, 144, 
           144, 144]
}

def plot_mask(mask):
    plt.plot(mask['az'], mask['el'], label=mask['name'], ls=':')

We can now visualize the colored blockage map

In [ ]:
plt.contourf(range(-180, 181), range(-5, 145), coverage_map, 100)
plt.colorbar()
plot_earth('2026-03-23', '2026-03-24', 'gray', '23 Mar 2026')
plot_earth('2026-04-06', '2026-04-07', 'orange', '6 Apr 2026')
plot_earth('2026-04-20', '2026-04-21', 'blue', '20 Apr 2026')
plot_mask(simplified)
plt.grid()
plt.show()

In [ ]:
coverage = coverage_map / np.sum(T) * 100
coverage[coverage == 0] = np.nan

fig = plt.figure(figsize=(14, 6))
plt.contourf(range(-180, 181), range(-5, 145), coverage, 20, alpha=0.1)
contour_lines = plt.contour(range(-180, 181), range(-5, 145), coverage, 20)
plt.clabel(contour_lines, contour_lines.levels, inline=True, fmt='%1.0f', fontsize=8)
plt.colorbar()
plt.ylim([0, 145])
plt.grid()
plt.ylabel('El [deg]')
plt.xlabel('Az [deg]')
plt.title('HGA disk percentage blockage')

plot_earth('2026-03-23', '2026-03-24', 'gray', '23 Mar 2026')
plot_earth('2026-04-06', '2026-04-07', 'orange', '6 Apr 2026')
plot_earth('2026-04-20', '2026-04-21', 'blue', '20 Apr 2026')

plot_mask(simplified)
plot_mask(astrium)

plt.legend()
plt.show()